In [6]:
import pandas as pd
from nexar_complete_with_validation import create_datasets_with_manual_split
from nexar_video_aug import create_video_transforms

In [8]:
df = pd.read_csv('df_encord_v3.csv').sample(150)
df.head()

,id,video_title,video_path,video_type,video_type_old,event_time_sec,original_time,split
16716,90cf7fb6a641c328cbe234571fa874f2_aug2,anonymized_90cf7fb6a641c328cbe234571fa874f2.mp4,../data/research-nvidia-data/nvidia-1/90cf7fb6...,collision,collision,20.594285,20.856962,train
14577,3bee26a218ab6ab7f096c214681ee8cc_dup,anonymized_3bee26a218ab6ab7f096c214681ee8cc.mp4,../data/research-nvidia-data/nvidia-1/3bee26a2...,normal,near_collision,12.071472,12.100000,train
5741,70700add526b7f2765575309c116e5e9_aug2,anonymized_70700add526b7f2765575309c116e5e9.mp4,../data/research-nvidia-data/nvidia-1/70700add...,collision,collision,21.041138,21.265085,train
7831,a871324f85d721038f34a7799380ac97,anonymized_a871324f85d721038f34a7799380ac97.mp4,../data/research-nvidia-data/nvidia-1/a871324f...,near_collision,near_collision,20.485810,19.950000,train
11334,df162362a14466ad26342788958f4c03_dup,anonymized_df162362a14466ad26342788958f4c03.mp4,../data/research-nvidia-data/nvidia-1/df162362...,normal,near_collision,10.926792,11.030000,train


In [9]:
train_ds, val_ds, test_ds = create_datasets_with_manual_split(
    metadata_df=df,
    transform_train=create_video_transforms(
        mode='train',
        enable_custom_augmentation=True,
        brightness_range=(0.9, 1.1),
        contrast_range=(0.9, 1.1),
        saturation_range=(0.9, 1.1),
        hue_range=(-0.05, 0.05),
        rotation_range=(-7, 7),
        scale_range=(0.95, 1.1),
        translate_range=(0.0, 0.07),
        grayscale_prob=0.02,
        blur_sigma=0.5,
        cutout_prob=0.1,
        cutout_count=(1, 2),
        cutout_size_range=(0.1, 0.15),
        horizontal_flip_prob=0.5,
        aug_probability=0.9
    ),
    transform_val=create_video_transforms(mode='val'),
    video_path_column='video_path',
    label_column='video_type', 
    id_column='id',
    split_column='split',
    sample_strategy='metadata_center',
    center_time_column='event_time_sec',
    fps=10,
    duration=5
)

train_viewer = train_ds.show_batch(m=3, rows_per_page=3)

Pre-computing FPS for metadata_center strategy...
Pre-computing FPS for metadata_center strategy...
Pre-computing FPS for metadata_center strategy...


In [11]:
val_viewer = val_ds.show_batch(m=3, rows_per_page=3)

In [14]:
batch_size = 12

loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0)

# Get a single batch
batch = next(iter(loader))

# Print batch information
print("Batch contents:")
print(f"- Frames: {batch['frames'].shape}, {batch['frames'].dtype}")
print(f"- Target: {batch['target']}")
print(f"- IDs: {batch['id']}")

Batch contents:
- Frames: torch.Size([12, 50, 224, 224, 3]), torch.float32
- Target: ['normal', 'collision', 'normal', 'normal', 'normal', 'normal', 'near_collision', 'normal', 'near_collision', 'normal', 'normal', 'normal']
- IDs: ['46f7f9467fc61c2a98ee9da7e2317342_dup', '3165df9a5754fbd20beaaf9cefd5eb33', '67fe4b68900c04889f8726cca480efbf_dup', '59f3d0479624a8928c4fde20808e5992_dup', '3a3c19858906c6a5784f9629727384b2_dup', '78ac2d6033fced98f6292463d9b25eb4_dup', '3d7aeb5f61194933d12afc9d550db776', 'e0661515db4963714f10518c08c39452', 'fea90afe789a84831bfe9246a4ed658b', '1f98cee6916347d1141636e289f71727', 'c12b2668-090b-4d4e-b507-8fb4e9eb3525', 'c4863600685f763d25fafdae039159aa_dup']
